In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("proc_date", "", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment_wmp
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH max_date AS (
    SELECT  DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AS max_calday
),
wmp AS (
    SELECT
        b.region_domain_vn AS mien,
        b.city,
        b.business_unit, 
        a.store_id, 
        a.product_id, 
        CASE 
            WHEN a.assortment_class_desc = 'CORE1' THEN 'Core 1' 
            WHEN a.assortment_class_desc = 'CORE2' THEN 'Core 2' 
            WHEN a.assortment_class_desc = 'CORE3' THEN 'Core 3' 
            ELSE a.assortment_class_desc 
        END AS core_range
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_his a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b
        ON a.store_id = b.store_id
    WHERE a.calday = (SELECT max_calday FROM max_date) 
    AND b.business_unit IN ('2000', '6000')
),
filtered_data AS (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY store_id, product_id 
               ORDER BY 
                   CASE
                       WHEN core_range = 'Core 1' THEN 1
                       WHEN core_range = 'Core 2' THEN 2
                       WHEN core_range = 'Core 3' THEN 3
                       ELSE 999
                   END
           ) AS rn
    FROM wmp
)
SELECT
mien,
city,
business_unit,
store_id,
product_id,
core_range
FROM 
filtered_data
WHERE rn = 1
""")